# SmolAgents Demo with vLLM Docker Server

This notebook demonstrates how to use [smolagents](https://github.com/huggingface/smolagents) - a barebones library for agents that think in code - with a shared vLLM Docker server.

## Prerequisites

1. Start the vLLM Docker container (shared by all team members):
   ```bash
   ./start_vllm_docker.sh
   ```

In [ ]:
import logging
from openai import OpenAI
from smolagents import CodeAgent, OpenAIServerModel, Tool

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("SmolAgents demo initialized!")


## DuckDuckGo Search Tool

Let's create a free DuckDuckGo search tool.


In [ ]:
from smolagents import DuckDuckGoSearchTool

duckduckgo_tool = DuckDuckGoSearchTool()

## Initialize the Agent

Now let's create a CodeAgent that can use both DuckDuckGo search and other tools. The agent will connect to our vLLM server running in Docker.

Note that if you want, you can also use smolagent's VLLMModel to launch an LLM from the python kernel https://huggingface.co/docs/smolagents/main/en/reference/models#smolagents.VLLMModel.

In [ ]:
# Initialize the OpenAI client to connect to vLLM server
client = OpenAI(
    api_key="EMPTY",  # vLLM doesn't require authentication
    base_url="http://localhost:8000/v1",
)
# Retrieve the model id from the vLLM server
try:
    model_ids = list(client.models.list())
    if model_ids:
        print("vLLM server is up and running!")
        print(f"Available models: {[model.id for model in model_ids]}")
    else:
        print("No models available")
except Exception as e:
    print(f"Error connecting to vLLM server: {e}")
    print("Make sure the server is running by executing `./start_vllm_docker.sh`")
model_id = model_ids[0].id


# Initialize the smolagents model to connect to our vLLM server
model = OpenAIServerModel(
    model_id=model_id,
    api_base="http://localhost:8000/v1",
    api_key="EMPTY",
    temperature=0.7
)

# Create the agent with DuckDuckGo search capability
agent = CodeAgent(
    tools=[duckduckgo_tool],
    model=model,
    stream_outputs=True
)

print("Agent initialized with DuckDuckGo search tool!")


## Test DuckDuckGo Search

Let's test our DuckDuckGo search tool first.


In [ ]:
# Test the DuckDuckGo search tool directly
test_query = "CRISPR gene editing 2024"
print(f"Testing DuckDuckGo search with query: '{test_query}'")
print("\n" + "="*60)

# Test the search tool directly
search_results = duckduckgo_tool(test_query)
print(search_results)

print("\n" + "="*60)
print("DuckDuckGo search test completed!")


## Drug Discovery Research

Let's try a drug discovery research task.


In [ ]:
# Drug discovery research task
drug_task = """
Research recent advances in AI-driven drug discovery. Focus on:
1. Machine learning models used for drug discovery
2. Recent successful drug candidates discovered using AI
3. Companies leading in this field
4. Challenges and limitations

Use web search to find specific examples and recent news.
"""

print("Starting drug discovery research with DuckDuckGo...")
print(f"Task: {drug_task.strip()}")
print("\n" + "="*60)

# Run the agent
result = agent.run(drug_task, max_steps=5)

print("\n" + "="*60)
print("Drug discovery research completed!")


## Custom Biology Tools

Let's create additional biology-specific search tools.


In [ ]:
class PubMedSearchTool(Tool):
    """Tool for searching PubMed using DuckDuckGo."""
    
    name = "search_pubmed"
    description = "Search PubMed for scientific articles related to biology topics"
    inputs = {
        "query": {
            "type": "string",
            "description": "The search query for PubMed articles"
        }
    }
    output_type = "string"
    
    def forward(self, query: str) -> str:
        """Search PubMed using DuckDuckGo site search."""
        search_query = f"site:pubmed.ncbi.nlm.nih.gov {query}"
        return duckduckgo_tool(search_query)

class BiorxivSearchTool(Tool):
    """Tool for searching arXiv using DuckDuckGo."""
    
    name = "search_biorxiv"
    description = "Search biorxiv for preprints related to biology topics"
    inputs = {
        "query": {
            "type": "string",
            "description": "The search query for biorxiv preprints"
        }
    }
    output_type = "string"
    
    def forward(self, query: str) -> str:
        """Search arXiv using DuckDuckGo site search."""
        search_query = f"site:biorxiv.org {query}"
        return duckduckgo_tool(search_query)

# Create the custom tools
pubmed_tool = PubMedSearchTool()
biorxiv_tool = BiorxivSearchTool()

# Create agent with multiple search tools
research_agent = CodeAgent(
    tools=[duckduckgo_tool, pubmed_tool, biorxiv_tool],
    model=model,
    stream_outputs=True
)

print("Research agent with multiple search tools created!")


In [ ]:
# Test the research agent with PubMed search
research_task = """
Find recent research papers about COVID-19 vaccine development using both web search and PubMed search.
Compare the results from both sources and provide a summary.
"""

print("Starting research with PubMed...")
print(f"Task: {research_task.strip()}")
print("\n" + "="*60)

# Run the research agent
result = research_agent.run(research_task, max_steps=5)

print("\n" + "="*60)
print("Research with PubMed completed!")
